In [1]:
import facebook
import requests
import json 

In [2]:
def get_page_credentials(company_name, page_name):
    with open("credentials.json", "r") as f:
        data = json.load(f)

    for company in data["companies"]:
        if company["company_name"] == company_name:
            for page in company["pages"]:
                if page["page_name"] == page_name:
                    return page["page_id"], page["access_token"]
            break
    return None, None

In [3]:
def post_media_link(page_name, media_url, caption, link_url, company_name):
    page_id, access_token = get_page_credentials(company_name, page_name)

    if page_id and access_token:
        api_url = f"https://graph.instagram.com/{page_id}/media"
        params = {
            "access_token": access_token,
            "media_url": media_url,
            "caption": f"{caption} More info: {link_url}"  # Include the link in the caption
        }

        response = requests.post(api_url, params=params)

        if response.status_code == 200:
            response_data = response.json()
            media_id = response_data.get("id")
            if media_id:
                print("Media with caption posted successfully.")
            else:
                print("Media post successful, but no media ID returned.")
        else:
            print("Media post failed:", response.status_code, response.text)
    else:
        print("Page not found or credentials not available.")

In [4]:
post_media_link("Instagram", "Image\peakpx.jpg", "Your caption here", "https://raisebe.com", "raisebe")

Media post failed: 400 Sorry, this content isn't available right now


In [5]:
import pandas as pd

def runallpost():
    df = pd.read_excel("posts.xlsx")
    filtered_rows = df.iloc[df['platform']=='Instagram']

    for index, row in filtered_rows.iterrows():
        page_name = row['Page Name']
        post_type = row['Post Type']

        media_url = row['Photo URL'] if pd.notnull(row['Photo URL']) else row['Video Path']

        if post_type == 'media':
            caption = row['Caption']
            post_instagram_media_with_link(page_name, media_url, caption)
        elif post_type == 'media_link':
            link_url = row['Link URL']
            link_caption = row['Link Caption']
            post_instagram_media_with_link(page_name, media_url, link_caption, link_url)
        else:
            print(f"Unknown post type '{post_type}' for {page_name}. Skipping post.")




In [6]:
def post_media_to_instagram(company_name, page_name, imagepath, caption):
    page_id, access_token = get_page_credentials(company_name, page_name)
    Graph_url = "https://graph.facebook.com/v17.0/{}/media".format(page_id)
    data = {
        "image_url": imagepath,
        "caption": caption,
        "access_token": access_token
    }

    r = requests.post(Graph_url, data=data)
    results = json.loads(r.text)

    if 'id' in results:
        creation_id = results['id']

        Publish_url = "https://graph.facebook.com/v17.0/{}/media_publish".format(page_id)
        data2 = {
            'creation_id': creation_id,
            "access_token": access_token
        }

        r = requests.post(Publish_url, data=data2)
        return r.text

In [7]:
post_media_to_instagram("raisebe", "Instagram", "Image\peakpx.jpg", "caption")